In [1]:
### Imports
import numpy as np
import pandas as pd
from warnings import filterwarnings
from tqdm import tqdm
import pickle

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, make_scorer
from sklearn.inspection import permutation_importance

In [2]:
### Filter warnings from numpy that show up occasionally during cv, should not affect performance
filterwarnings("ignore", message="invalid value encountered in cast", category=RuntimeWarning)

In [3]:
### Parameters
# random seed
# allows for reproducibility
# default is 42, which is used in publication
random_seed = 42

# number of iterations of random search to use to optimize each model's hyperparameters in validation
# increasing will improve results but increase computation time
# requires under 3 minutes for every 20 iterations with the following system specs:
# Processor	Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, 2112 Mhz, 4 Core(s), 8 Logical Processor(s)
# Installed Physical Memory (RAM)	16.0 GB
# in publication, 192 iterations are used, requiring < 30 minutes total
random_search_iter = 192

# number of iterations used in calculations of feature importance via permutation importance
# increasing will decrease variance in feature imoprtance results but increase computation time
# for publication 20 is used, taking approximately 10 minutes to run
feature_importance_iter = 20

# which metric to prioritize in model training
# AUC evaluates threshold independent ability for a model to make decisions,
# so it is a good metric to optimize before a model is deployed for 
# clinical settings.
metric_to_prioritize = 'auc_roc_ovo'

In [4]:
### Model setup (including parameter distributions for validation)
model_names = ["RandomForestClassifier", "KNeighborsClassifier"]
model_abbreviations = ["RF", "kN"]
# Chat GPT helped to refine the parameter grid
param_dists = [
    {   # Random Forest                                                                         # options for...
        "classify":                         [RandomForestClassifier(random_state=random_seed)], # model being trained (Random Forest)                                                             # options for...
        "reduce_dim__n_components":         [5, 10, None],                                      # number of components to keep from PCA
        "classify__n_estimators":           [100, 200, 300],                                    # number of decision trees in the random forest
        "classify__max_depth":              [10, 20, 30, None],                                 # maximum allowed depth of the decision trees
        "classify__max_features":           ["sqrt", "log2"],                                   # restriction on number of features considered per split
        "classify__bootstrap":              [True],                                             # whether to bootstrap
        "classify__min_samples_split":      [2, 3, 4],                                          # minimum number of samples required to split a node
        "classify__min_samples_leaf":       [1, 2, 4]                                           # minimum number of samples required at a leaf
    },
    {   # K nearest neighbors                                                                   # options for...
        "classify":                         [KNeighborsClassifier()],                           # model being trained (K nearest neighbors)
        "reduce_dim__n_components":         [5, 10, None],                                      # number of components to keep from PCA
        "classify__weights":                ["uniform", "distance"],                            # vote weighting method
        "classify__p":                      [1, 1.5, 2, 2.25],                                  # exponent parameter for minkowski distance
        "classify__n_neighbors":            [3, 5, 7, 11, 15, 19, 23, 25],                      # number of neighbors considered
    }
]
model_options = dict(zip(model_names, param_dists))
model_abbreviations = dict(zip(model_names, model_abbreviations))

In [5]:
### Load data
# extract data from file
excel_file = "../Data/Cell_Data.xlsx"
sheets_dict = pd.read_excel(excel_file, sheet_name = None) # dict where each value is a data frame (sheet)

# cell type labels are sheet labels in the excel document and keys in the df dict
# cell_types = ['L1', 'L2', 'L3', 'Monoblasts', 'Myeloblasts', 'Reactive Lymphs']
cell_types = list(sheets_dict.keys())

for cell_type in cell_types:
    # add a cell type column to each data frame
    sheets_dict[cell_type]["cell_type"] = cell_type

# build one singular df with a class column identifying cell_type
df_list = [sheets_dict[cell_type] for cell_type in cell_types]
combined_df = pd.concat(df_list, ignore_index=True)

# throw out name of image, area, and total image area
combined_df.drop(["Image", "Area (microns^2)", "TotalImageArea"], axis=1, inplace=True)

In [6]:
### Summarize data
# show dataframe
display(combined_df)
# show unique cell types
print("Cell types:", set(combined_df["cell_type"]))

,Lacunarity,Total Length (microns),Endpoints,HGU (microns),Branchpoints,Box-Counting Fractal Dimension,Curvature_50.0,% High Density Matrix,Alignment,Branchpoints/Total Length,Endpoints/Total Length,Average Fiber Length,Average Fiber Thickness,cell_type
0,79.211,4005,69,58.043,367,1.175,26.812,0.587,0.11620,0.091635,0.017228,18.371560,14.656679,L1
1,110.888,3485,90,38.722,227,1.112,36.987,0.558,0.06317,0.065136,0.025825,21.987382,16.011478,L1
2,173.399,1880,43,43.721,59,1.024,30.577,0.542,0.09171,0.031383,0.022872,36.862745,28.829787,L1
3,96.193,3203,59,54.288,161,1.115,24.991,0.622,0.03657,0.050265,0.018420,29.118182,19.419294,L1
4,132.483,2414,54,44.704,82,1.088,26.482,0.575,0.05596,0.033969,0.022370,35.500000,23.819387,L1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,86.239,3993,99,40.333,80,1.110,34.130,0.655,0.03727,0.020035,0.024793,44.614525,16.403706,Reactive Lymphs
2596,124.097,2777,61,45.525,88,1.097,39.728,0.552,0.10080,0.031689,0.021966,37.275168,19.877566,Reactive Lymphs
2597,76.126,3913,65,60.200,126,1.148,33.045,0.745,0.07569,0.032200,0.016611,40.973822,19.039100,Reactive Lymphs
2598,75.398,4100,58,70.690,138,1.170,27.441,0.739,0.03438,0.033659,0.014146,41.836735,18.024390,Reactive Lymphs


Cell types: {'Monoblasts', 'Myeloblasts', 'L1', 'L2', 'Reactive Lymphs', 'L3'}


In [7]:
### Construct the data permutations
# now make different combinations of the data
# L1 and RL
l1_and_rl_df = combined_df[combined_df["cell_type"].isin(["L1", "Reactive Lymphs"])]
# L2 and RL
l2_and_rl_df = combined_df[combined_df["cell_type"].isin(["L2", "Reactive Lymphs"])]
# L3 and RL
l3_and_rl_df = combined_df[combined_df["cell_type"].isin(["L3", "Reactive Lymphs"])]
# L1, L2, RL
l1_and_l2_and_rl_df = combined_df[combined_df["cell_type"].isin(["L1", "L2", "Reactive Lymphs"])]
# Monoblast and RL
monoblast_and_rl_df = combined_df[combined_df["cell_type"].isin(["Monoblasts", "Reactive Lymphs"])]
# Myeloblast and RL
myeloblast_and_rl_df = combined_df[combined_df["cell_type"].isin(["Myeloblasts", "Reactive Lymphs"])]
# Combined blasts and RL
combined_blasts_and_rl_df = combined_df.copy()
combined_blasts_and_rl_df.loc[combined_df["cell_type"] != "Reactive Lymphs", "cell_type"] = "Blasts"

# save dataframes in a dictionary with keys as their labels
list_of_df = [combined_df, l1_and_rl_df, l2_and_rl_df, l3_and_rl_df, l1_and_l2_and_rl_df, monoblast_and_rl_df, myeloblast_and_rl_df, combined_blasts_and_rl_df]
list_of_df_names = ["All Cells", "L1 and RL", "L2 and RL", "L3 and RL", "L1 and L2 and RL", "Monoblast and RL", "Myeloblast and RL", "All blasts and RL"]
dict_of_df = dict(zip(list_of_df_names, list_of_df))

In [8]:
### Define pipeline
# first, scale the data with a standard scaler
# then, reduce dimensionality with PCA (number of dimensions tbd in model selection)
# finally, use either random forest or knn to classify (also tbd)
# help from: https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html#sphx-glr-auto-examples-compose-plot-compare-reduction-py

# pass the selection of the classification algorithm through to validation
pipe = Pipeline(
    [
        ("scaling", StandardScaler()),
        ("reduce_dim", PCA(random_state=random_seed)),
        ("classify", "passthrough")
    ]
)

In [9]:
### Define custom model evaluation metrics
def custom_success_metrics(y_test, y_pred, type):
    # find confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    # for each class, evaluate true positives, true negatives, etc (vectorized)
    # Chat GPT helped here
    tps = np.diag(cm)
    fps = np.sum(cm, axis=0) - tps
    fns = np.sum(cm, axis=1) - tps
    tns = np.sum(cm) - (fps + fns + tps)
    # each of these should be a vector with index corresponding to class
    # now return a custom dictionary of balanced (macro-averaged) versions of accuracy, precision, sensitivity, specificity
    # treat nan values as a score of 0, most likely are caused by empty classes
    # also silence warnings caused by these issues during cv
    # ChatGPT helped generate code to sanitize the final value and suppress warnings
    val = None
    with np.errstate(divide='ignore', invalid='ignore'):            
        match type:
            case "accuracy": val = np.nanmean((tps + tns) / (tps + tns + fns + fps))
            case "precision": val = np.nanmean(tps / (tps + fps))
            case "sensitivity": val = np.nanmean(tps / (tps + fns))
            case "specificity": val = np.nanmean(tns / (tns + fps))
            case _: raise ValueError(f"Type \"{type}\" is not supported")
    return 0.0 if np.isnan(val) else val

# metrics to be calculated on each proposed model during cross validation (for model selection and hyperparameter optimization)
metrics_dict = {
    "accuracy": make_scorer(lambda ytest, ypred: custom_success_metrics(ytest, ypred, "accuracy")),
    "precision": make_scorer(lambda ytest, ypred: custom_success_metrics(ytest, ypred, "precision")),
    "sensitivity": make_scorer(lambda ytest, ypred: custom_success_metrics(ytest, ypred, "sensitivity")),
    "specificity": make_scorer(lambda ytest, ypred: custom_success_metrics(ytest, ypred, "specificity")),
    # use ovo and ovr for comparison
    "auc_roc_ovo": make_scorer(roc_auc_score, multi_class = "ovo", response_method=["predict_proba", "decision_function"]),
    "auc_roc_ovr": make_scorer(roc_auc_score, multi_class = "ovr", response_method=["predict_proba", "decision_function"]),
    "f1": "f1_macro"
}

# Also define by class for use in final testing
def custom_success_metrics_by_class(cm):
    # uses confusion matrix
    # for each class, evaluate true positives, true negatives, etc (vectorized)
    # Chat GPT helped to get these straight
    tps = np.diag(cm)
    fps = np.sum(cm, axis=0) - tps
    fns = np.sum(cm, axis=1) - tps
    tns = np.sum(cm) - (fps + fns + tps)
    # each of these should be a vector with index corresponding to class
    # now compute the vector for either accuracy, precision, sensitivity or specificity
    values = {
        "accuracy"      : (tps + tns) / (tps + tns + fns + fps),
        "precision"     : tps / (tps + fps),
        "sensitivity"   : tps / (tps + fns),
        "specificity"   : tns / (tns + fps)
    }
    return values

In [10]:
### Define replicable train-test split
def ttsplit(df, random_state):
    # split the data stratified by cell type 70-30 for training-testing
    # use a fixed random state so that each split can be replicated exactly
    # our response variable is cell type
    y_data = df["cell_type"]
    X_data = df.drop("cell_type", axis=1, inplace=False)
    # split our data points into 70-30 training and testing sets, stratified by cell type
    # returns X_train, X_test, y_train, y_test
    return train_test_split(X_data, y_data, test_size = 0.3, random_state=random_state, shuffle=True, stratify=y_data)

In [11]:
def model_validation(X_train, y_train, param_dist):
    # set up cross validation and randomized search
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
    # prioritize accuracy in training
    best_model_indicator = 'rank_test_' + metric_to_prioritize
    best_model_selector = lambda cv_results : np.argmin(cv_results[best_model_indicator])
    search = RandomizedSearchCV(pipe, n_iter=random_search_iter, n_jobs=-1, param_distributions=param_dist, scoring=metrics_dict, refit=best_model_selector, cv=cv, verbose=False, error_score='raise', random_state=random_seed)
    # cross validate
    search.fit(X_train, y_train)
    # save and return the validation results for this specific classification job
    best_score_ind = np.argmin(search.cv_results_['mean_test_' + metric_to_prioritize])
    best_score_mean = search.cv_results_['mean_test_' + metric_to_prioritize][best_score_ind]
    best_score_std = search.cv_results_['std_test_' + metric_to_prioritize][best_score_ind]
    results = pd.DataFrame(search.cv_results_)
    results.sort_values(by=best_model_indicator, inplace=True)
    validation_results = {
        'results_df'                :   results,
        'results'                   :   search.cv_results_,
        'best_params'               :   search.best_params_,
        'best_score_mean'           :   best_score_mean,
        'best_score_std'            :   best_score_std,
        'validation_set_X'          :   X_train,
        'validation_set_y'          :   y_train,
        'validation_n_iter'         :   random_search_iter,
        'param_dist'                :   param_dist,
    }
    return validation_results

In [12]:
### How to unpickle models
# # (do this if you don't want to train models yourself)
# with open("results.pickle", "rb") as file:
#     all_results = pickle.load(file)
# # Check to make sure the filename is the right one.
# # Comment out the cell below and run all after uncommenting this cell

In [13]:
### Model validation and training

# Perform random search for model selection and hyperparameter optimization
# Run random search on the pipeline for each permutation of the dataset stored in dict_of_df
# save results and info for each model in a dictionary
all_results = dict()

# create training jobs for each combination of classification task and model
jobs = list()
for name, df in dict_of_df.items():
    for model_name in model_options.keys():
        jobs.append((name, df, model_name))
        all_results[(name, model_name)] = dict()
        all_results[(name, model_name)]['dataframe'] = df

# run all jobs with progress bar
jobs = tqdm(jobs)
for name, df, model_name in jobs:
    # identify which permutation is being run
    jobs.set_description(f"Randomized Search CV on {name} data with {model_name}")
    # record results for this specific permutation in results_dict
    results_dict = dict()
    # split the data
    X_train, X_test, y_train, y_test = ttsplit(df, random_seed)
    # cross validate / randomized search
    param_dist = model_options[model_name]
    validation_results = model_validation(X_train, y_train, param_dist)
    # save results
    all_results[(name, model_name)].update({
        'validation_results'    :   validation_results,
        'train_test_split'      :   (X_train, X_test, y_train, y_test)
    })

# write the validation results dataframes to an excel sheet
with pd.ExcelWriter("validation.xlsx", engine='openpyxl') as writer:
    for (name, model_name), results_dict in all_results.items():
        results = results_dict['validation_results']['results_df']
        results.to_excel(writer, sheet_name=name + "_" + model_abbreviations[model_name], index=False)

# also pickle the all_results to get later if needed
with open("results.pickle", "wb") as file:
    pickle.dump(all_results, file)

Randomized Search CV on All Cells data with RandomForestClassifier:   0%|          | 0/16 [00:00<?, ?it/s]

Randomized Search CV on All blasts and RL data with KNeighborsClassifier: 100%|██████████| 16/16 [24:39<00:00, 92.48s/it]   


In [14]:
### Model testing and recording results
# Test best performing models and report output in human readable format
# Iterate through each group of labels for which a model was trained (all cells, L1 vs RL, etc)
# Evaluate both k-nearest neighbors and random forest models on the final holdout / test set
# Save information in results_<date>.txt and in results.pickle

custom_metrics = ['accuracy', 'precision', 'sensitivity', 'specificity']
# for feature importance
metrics_dict_fi = {metric : make_scorer(lambda ytest, ypred: custom_success_metrics(ytest, ypred, metric)) for metric in custom_metrics}
# these models will be trained with the best parameters selected in cross validation

with open("results.txt", "w") as file:
    for (name, model_name), results_dict in all_results.items():
        ### Train the model selected in validation (highest accuracy) on the entire validation set
        # extract model parameters
        model = model_options[model_name]['classify'][0]
        best_estimator_params = results_dict['validation_results']['best_params']
        best_estimator = Pipeline(
            [
                ("scaling", StandardScaler()),
                ("reduce_dim", PCA()),
                ("classify", clone(model))
            ]
        )

        # train the model
        X_train, X_test, y_train, y_test = results_dict['train_test_split']
        best_estimator.set_params(**best_estimator_params)
        best_estimator.fit(X_train, y_train)

        ### Test model on the resulting dataset
        # get the best model found for the permutation given judging by accuracy
        # find model predicted values and probabilities on the testing data
        y_pred = best_estimator.predict(X_test)
        y_prob = best_estimator.predict_proba(X_test)
        
        ### Compute performance metrics
        # compute the confusion matrix
        cm = confusion_matrix(y_test, y_pred, normalize=None)
        
        # evaluate and report model accuracy, precision, sensitivity, specificity
        metric_vals_dict = custom_success_metrics_by_class(cm)
        
        # evaluate Area Under the Curve
        # try both one-vs-one and one-vs-rest strategy for multi-class situations
        if len(best_estimator.classes_) > 2:
            # is multi-class classifier
            ovo = roc_auc_score(y_test, y_prob, multi_class='ovo')
            ovr = roc_auc_score(y_test, y_prob, multi_class='ovr')
            metric_vals_dict['ROC AUC (OVO)'] = ovo
            metric_vals_dict['ROC AUC (OVR)'] = ovr

        else:
            # is binary classifier
            auc = roc_auc_score(y_test, y_prob[:,1])
            metric_vals_dict['ROC AUC'] = auc

        ### Compute feature importance (permutation-based importances)
        # on training set
        pi_dict_train = permutation_importance(best_estimator, X_train, y_train, scoring=metrics_dict_fi, n_repeats=feature_importance_iter, random_state=random_seed)
        # on test set
        pi_dict_test = permutation_importance(best_estimator, X_test, y_test, scoring=metrics_dict_fi, n_repeats=feature_importance_iter, random_state=random_seed)
        # find averaged permutation-based feature importances
        # final importance score for each feature is the balanced average of 8 values: the accuracy, precision, sensitivity and specificity permutation importances on train set and on test set
        feature_vals = list()
        for bunch in [pi_dict_train, pi_dict_test]:
            for metric in custom_metrics:
                feature_vals.append(bunch[metric]['importances_mean'])
        feature_vals = np.stack(feature_vals)
        # compute balanced average
        feature_means = np.mean(feature_vals, axis=0)
        # collect these means into a final feature importance dict
        features = list(combined_df.columns)
        feature_means_dict = dict(zip(features, feature_means))
        # sort by final importance for output in the summary
        feature_importance_list = sorted(feature_means_dict.items(), key=lambda k: feature_means_dict[k[0]], reverse=True)
        
        ### Write results summary to results.txt
        file.write(f'*{name}*, {model_name}\n')
        file.write(f'classes:{str(best_estimator.classes_)}\n')
        file.write(f'confusion matrix:\n{str(cm)}\n')
        for metric, met_vals in sorted(metric_vals_dict.items(), key=lambda x: x[0]):
            if metric in custom_metrics:
                file.write(f'{metric} by class: {str(met_vals)}\n')
                file.write(f'average {metric} (balanced, by class): {str(np.mean(met_vals))}\n')
            else:
                file.write(f'{metric}: {str(met_vals)}\n')
        file.write(f"Features: {features}\n")
        file.write("Features in order of final importance:\n")
        for i, pair in enumerate(feature_importance_list):
            feature, importance = pair
            dot = "."
            file.write(f"\t{str(i+1)+dot:<5} {feature:<35} {importance}\n")
        file.write("Model details:\n")
        file.write(str(results_dict['validation_results']['best_params']) + '\n')
        file.write('\n\n')

        ### Add results to results_dict dictionary (located within all_results) to pickle later
        results_dict["final_results"] = {
            'trained_model'                         :   best_estimator,
            'model_params'                          :   best_estimator_params,
            'y_true'                                :   y_test,
            'y_predicted'                           :   y_pred,
            'y_predicted_probabilities'             :   y_prob,
            'confusion_matrix'                      :   cm,
            'metric_values'                         :   metric_vals_dict,
            'classes'                               :   best_estimator.classes_,
            'permutation_importance_n_repeats'      :   feature_importance_iter,
            'permutation_importance_train'          :   pi_dict_train,
            'permutation_importance_test'           :   pi_dict_test,
            'feature_importance_final_importances'  :   feature_means_dict
        }

### Write the pickled all_results to get them later if needed
with open("results.pickle", "wb") as file:
    pickle.dump(all_results, file)